In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression, LogisticRegression
from scipy.stats import pearsonr
import statsmodels.api as sm
!pip3 install --upgrade openpyxl

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [3]:
def correlation_pval(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns = df.columns)
    corr = dfcols.transpose().join(dfcols, how = 'outer')
    pval = dfcols.transpose().join(dfcols, how = 'outer')
    for r in df.columns:
        for c in df.columns:
            corr[r][c], pval[r][c] = pearsonr(df[r], df[c])
            #for correlation considered 0.7
            if r!=c and (corr[r][c] > 0.7 or corr[r][c] < -0.7):
                print('correal')
                print(r, c, corr[r][c])
            #for pval considered 0.10
            if r!=c and (pval[r][c] > 0.10 or pval[r][c] < -0.10):
                print('pval')
                print(r, c, pval[r][c])
    return corr, pval

In [4]:
logis_df = pd.read_excel('loan.xlsx')
print(logis_df.head())
print(logis_df.columns)

columns_sel = ['Sex','Res_status','Telephone','Occupation','Job_status','Liab_ref','Acc_ref','Decision']
for var in columns_sel:
    cat_df = pd.get_dummies(logis_df[var], prefix=var)
    #print(cat_df)
    logis_df = logis_df.join(cat_df)

to_keep = [i for i in logis_df.columns if i not in columns_sel]
logis_df = logis_df[to_keep]
print(len(to_keep))
print(logis_df.columns)

data_corr, data_pval = correlation_pval(logis_df)
print(data_corr)
print(data_pval)

#first set of drops
logis_df = logis_df.drop('Decision_reject', axis=1)
logis_df = logis_df.drop('Liab_ref_f', axis=1)
logis_df = logis_df.drop('Telephone_not_given', axis=1)
logis_df = logis_df.drop('Res_status_owner', axis=1)
logis_df = logis_df.drop('Res_status_rent', axis=1)
logis_df = logis_df.drop('Sex_M', axis=1)
logis_df = logis_df.drop('Acc_ref_oth_inst_', axis=1)
logis_df = logis_df.drop('Job_status_unemploye', axis=1)

print(len(logis_df.columns))

#second set of drops
logis_df = logis_df.drop('Job_status_student', axis=1)
logis_df = logis_df.drop('Job_status_private_s', axis=1)
logis_df = logis_df.drop('Job_status_military', axis=1)
logis_df = logis_df.drop('Occupation_productio', axis=1)
logis_df = logis_df.drop('Occupation_creative_', axis=1)
logis_df = logis_df.drop('Occupation_manager', axis=1)
logis_df = logis_df.drop('Occupation_sales', axis=1)
logis_df = logis_df.drop('Occupation_executive', axis=1)
logis_df = logis_df.drop('Liab_ref_t', axis=1)

data_corr, data_pval = correlation_pval(logis_df)
print(data_corr)
print(data_pval)

x_df = logis_df.drop('Decision_accept', axis=1)
y_df = logis_df['Decision_accept']
print(x_df)
print(y_df)

scaler = preprocessing.StandardScaler().fit(x_df)
x_scaled = scaler.transform(x_df)
fin_reg = LogisticRegression(max_iter = 500).fit(x_scaled, y_df)
print('R^2 score: before final', fin_reg.score(x_scaled, y_df))

x = np.array(x_df)
print(x.shape)
y = np.array(y_df)
print(y.shape)

  Sex        Age  Time_at_address Res_status  Telephone Occupation Job_status  \
0   M  50.750000            0.585      owner      given  unemploye  unemploye   
1   M  19.670000           10.000       rent  not_given   labourer  governmen   
2   F  52.830002           15.000      owner      given  creative_  private_s   
3   M  22.670000            2.540       rent  not_given  creative_  governmen   
4   M  29.250000           13.000      owner      given     driver  governmen   

   Time_employed  Time_bank Liab_ref Acc_ref  Home_Expn  Balance Decision  
0              0          0        f   given        145        0   reject  
1              0          0        t   given        140        0   reject  
2              5         14        f   given          0     2200   accept  
3              2          0        f   given          0        0   accept  
4              0          0        f   given        228        0   reject  
Index(['Sex', 'Age', 'Time_at_address', 'Res_status', 'Te

In [5]:
scaler = preprocessing.StandardScaler().fit(x)
x_scaled = scaler.transform(x)

logi_reg = LogisticRegression().fit(x_scaled, y)
print("R square ", logi_reg.score(x_scaled, y))

R square  0.7948717948717948
